## Urgent Data 
> take 2.0

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from doubledot import *
import pandas as pd
from time import sleep
import os

In [23]:
sf = Salesforce()

Directory 'atms_download' already exists.


In [24]:
### Clean out old data ?
# for table in Salesforce.model_d.keys():
#     sf.delete_sf_objects(table)

In [27]:
#| eval: false
# 415645 73012 105765 and 342765 are also good contacts to bring in when you are rea and organization 133156
pelton_ids = [ 4708, 119430, 119431, 144164,144165, 144166, 144167, 415645, 73012, 105765, 342765 ]
sf.retrieve_atms_records_by_contactId(pelton_ids)

# number of objects in dictionary
assert len(sf.atms.obj_d) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of objects in dictionary {sf.atms.obj_d.keys()}"
# number of files in directory
assert len(os.listdir(sf.atms.download_dir)) == len(['sales', 'contacts', 'memberships','items']), f"wrong number of files in directory {sf.atms.download_dir}"



Directory 'atms_download' already exists.
Directory 'atms_download' already exists.
my id is 6s3x7kgs
144164
we have contact_id 144164 and obj is sales
http://crm-api-telus.atmsplus.com/api/sales/contact/144164
[{'saleKey': '310928', 'saleAmount': '123.0500', 'paymentAmount': '123.0500', 'saleDate': '2005-08-19T10:23:33.03', 'active': True, 'terminalKey': 4, 'ticketCount': 0, 'eventDate': None, 'booking': {'bookingId': 48112, 'bookingContactKey': 192717, 'bookingContactType': 'Primary     ', 'contactKey': 144164, 'contactIndividualKey': 137810, 'contactOrganizationKey': 5649, 'displayName': 'Pelton, Donna', 'firstName': 'Donna', 'lastName': 'Pelton', 'email': 'tdpelton@shaw.ca', 'phone': '7804592956'}, 'saleComment': None, 'saleDetails': [{'saleDetailId': 760379, 'saleId': 310928, 'itemKey': 7, 'scheduleKey': None, 'rateKey': 3, 'categoryKey': None, 'itemQuantity': 1, 'pricingPriceKey': 207, 'itemPrice': 105.0, 'itemTotal': 105.0, 'couponTotal': 0.0, 'discountTotal': 0.0, 'total': 123.

In [ ]:
#| eval : false
_d = sf.perfect_data()
for k, v in _d.items():
    display(v)

Salesforce.process_objects: sf_object_s : Contact
Salesforce.process_objects: sf_object_s : Membership__c
Salesforce.process_objects: sf_object_s : MembershipTerm__c
Salesforce.process_objects: sf_object_s : MembershipMember__c
Salesforce.process_objects: sf_object_s : Sale__c
Salesforce.process_objects: sf_object_s : SaleDetail__c
Salesforce.process_objects: sf_object_s : Ticket__c
this should fail
and it did. good.
dropping duplicates for  Contact  on  contactId__c ...
dropping duplicates for  Membership__c  on  membershipId__c ...
dropping duplicates for  MembershipTerm__c  on  membershipTermId__c ...
dropping duplicates for  MembershipMember__c  on  membershipMemberId__c ...
dropping duplicates for  Sale__c  on  saleId__c ...
dropping duplicates for  SaleDetail__c  on  saleDetailId__c ...
dropping duplicates for  Ticket__c  on  ticketId__c ...
Contact
Membership__c
MembershipTerm__c
MembershipMember__c
Sale__c
SaleDetail__c
Ticket__c
this should NOT fail
finally, it did not fail. G

,LastName,FirstName,MailingPostalCode,MailingCity,MailingStreet,MailingCountry,Phone,Email,contactId__c
0,Pelton,Donna,T8N 2L8,St. Albert,96 Aspen Crescent,Canada,7804273658,,144164
1,Pelton,Donalda,T5Z 2A6,Edmonton,17607 93 Street NW,Canada,4034529100,,4708
2,Pelton,Trevor,T8N 2L8,St. Albert,96 Aspen Crescent,Canada,7804273658,,144165
3,Pelton,Dan,T5Z 2A6,Edmonton,17607 93 Street NW,Canada,4034529100,,119430
4,Pelton,Evan,T5Z 2A6,Edmonton,17607 93 Street NW,Canada,4034529100,,119431
5,Pelton,Brittany,,St. Albert,,Canada,7804273658,,144166
6,Pelton,Garion,,St. Albert,,Canada,7804273658,,144167
7,Van Os,Adrian,,Edmonton,,Canada,,,105765
8,Van Os,Adrian,T5K 2T3,Edmonton,10119-97 A Avenue,Canada,7807184261,,342765
9,Mann,Joseph,T6E 2P9,Edmonton,9704 88 Avenue NW,Canada,,,73012


,membershipId__c,memberSince__c,updateDate__c
0,12110,2005-08-19T10:24:00.67,2005-08-19T10:19:24.647
1,781,1997-04-22T00:00:00,2001-06-18T12:07:00.92
2,8633,2002-11-29T14:48:53.857,2002-11-29T14:46:12.95
3,48042,1997-04-22T00:00:00,2017-12-19T13:16:17.07
11,6351,2001-05-30T00:00:00,2001-06-18T12:07:00.92
12,2929,1999-01-08T00:00:00,2001-06-18T12:07:00.92
13,51485,2018-10-24T00:00:00,2018-10-24T14:54:04.59
14,55236,2019-05-04T00:00:00,2019-05-04T12:59:49.46


,membershipTermId__c,membershipKey__r.membershipId__c,effectiveDate__c,expiryDate__c,membershipType__c,upgradeFromTermKey__c,giftMembership__c,refunded__c,saleDetailKey__c,itemKey__c
0,27514,12110,2005-08-19T00:00:00,2006-08-19T23:59:59,Family Membership,NaN,NaN,NaN,760379.0,7
3,8776,781,2001-01-10T00:00:00,2001-12-31T23:59:59,Staff Over 600 Hours,NaN,NaN,NaN,NaN,84
19,91626,781,2017-12-19T00:00:00,2017-12-31T23:59:59,Staff Over 600 Hours,NaN,NaN,NaN,5260354.0,84
20,102412,781,2019-03-20T00:00:00,2050-01-01T23:59:59,Lifetime Membership,NaN,NaN,NaN,5874240.0,1606
21,21115,8633,2002-11-29T00:00:00,2003-11-28T23:59:59,Family Membership,NaN,NaN,NaN,297233.0,7
22,46500,8633,2010-02-04T00:00:00,2021-06-30T23:59:59,Family Membership,NaN,NaN,NaN,1737719.0,7
23,91624,48042,2018-01-01T00:00:00,2018-12-31T23:59:59,Multiple IMAX Fanatic Annual Movie Passes,NaN,NaN,NaN,5260349.0,2594
56,11895,6351,2001-05-30T00:00:00,2002-05-31T23:59:59,Family Membership,NaN,NaN,NaN,NaN,7
57,20076,6351,2002-06-12T00:00:00,2003-06-11T23:59:59,Family Membership,NaN,NaN,NaN,208524.0,7
58,24459,6351,2004-04-20T00:00:00,2005-04-20T23:59:59,Family Membership,NaN,NaN,NaN,523900.0,7


,membershipMemberId__c,membershipTermKey__r.membershipTermId__c,cardNumber__c,membershipNumber__c,cardStatus__c,contactKey__r.contactId__c,displayName__c
0,55676,27514,1,1211001,Active,144164,"Pelton, Donna"
1,55677,27514,2,1211002,Active,144165,"Pelton, Trevor"
2,55678,27514,3,1211003,Active,144166,"Pelton, Brittany"
3,55679,27514,4,1211004,Active,144167,"Pelton, Garion"
12,31403,8776,1,78101,Active,4708,"Pelton, Donalda"
30,243277,102412,4,78104,Active,119430,"Pelton, Dan"
31,44932,21115,1,863301,Active,4708,"Pelton, Donalda"
32,44933,21115,2,863302,Active,119430,"Pelton, Dan"
33,44934,21115,3,863303,Active,119431,"Pelton, Evan"
37,171883,91624,1,4804201,Active,4708,"Pelton, Donalda"


,saleId__c,saleAmount__c,paymentAmount__c,saleDate__c,active__c,terminalKey__c,booking_bookingId__c,booking_bookingContactKey__c,booking_contactKey__r.contactId__c,booking_contactIndividualKey__c,booking_contactOrganizationKey__c,booking_displayName__c,booking_firstName__c,booking_lastName__c,booking_email__c,booking_phone__c
68,889832,0.00,0.00,2012-02-23T15:10:10.667,True,30,147465,524016,4708,3,NaN,"Pelton, Donalda",Donalda,Pelton,ddpelton@icloud.com,7.804769e+09
72,1083351,24.33,24.33,2013-09-01T14:50:28.317,True,2,221770,627466,4708,3,NaN,"Pelton, Donalda",Donalda,Pelton,ddpelton@icloud.com,7.804769e+09
73,1512308,14.65,14.65,2017-01-27T10:11:54.13,True,4,375821,861086,4708,3,NaN,"Pelton, Donalda",Donalda,Pelton,ddpelton@icloud.com,7.804769e+09
74,1598590,13.61,13.61,2017-10-14T09:30:37.19,True,4,415524,912956,4708,3,NaN,"Pelton, Donalda",Donalda,Pelton,ddpelton@icloud.com,7.804769e+09
77,1615641,0.00,0.00,2017-12-19T13:21:29.843,True,1,424943,925093,4708,3,NaN,"Pelton, Donalda",Donalda,Pelton,ddpelton@icloud.com,7.804769e+09
98,82192,19.94,19.94,2002-06-12T11:52:36.827,True,9,10815,43343,73012,68317,NaN,"Mann, Joseph",Joseph,Mann,pubmann@shaw.ca,7.804387e+09
101,323674,35.14,35.14,2005-10-24T15:58:15.573,True,9,50870,203813,73012,68317,NaN,"Mann, Joseph",Joseph,Mann,pubmann@shaw.ca,7.804387e+09
102,422399,116.60,116.60,2007-02-27T14:10:36.45,True,3,69090,276895,73012,68317,NaN,"Mann, Joseph",Joseph,Mann,pubmann@shaw.ca,7.804387e+09
103,450945,32.07,32.07,2007-06-17T14:10:09.673,True,8,73727,295451,73012,68317,NaN,"Mann, Joseph",Joseph,Mann,pubmann@shaw.ca,7.804387e+09
105,1718108,0.00,0.00,2018-10-26T15:43:22.293,True,55,477622,992271,415645,405442,NaN,"Vargo, Erica",Erica,Vargo,ericalinvargo@gmail.com,7.807176e+09


,saleDetailId__c,itemKey__c,scheduleKey__c,rateKey__c,categoryKey__c,itemCategory__c,pricingPriceKey__c,itemPrice__c,itemTotal__c,couponTotal__c,discountTotal__c,total__c,revenueDate__c,refundReason__c,refundReasonKey__c,systemPriceOverride__c,membershipTermKey__r.membershipTermId__c,saleId__r.saleId__c
292,3612729,940,352648.0,1,1.0,NaN,13402,6.98,6.98,6.98,NaN,NaN,2013-09-01T15:00:00,NaN,NaN,N,46500.0,1083351
293,3612730,940,352648.0,1,1.0,NaN,13402,6.98,6.98,6.98,NaN,NaN,2013-09-01T15:00:00,NaN,NaN,N,46500.0,1083351
294,3612731,940,352648.0,1,1.0,NaN,13438,10.46,10.46,NaN,NaN,10.98,2013-09-01T15:00:00,NaN,NaN,N,46500.0,1083351
295,3612743,1,333788.0,1,1.0,NaN,13836,NaN,NaN,NaN,NaN,NaN,2013-09-01T10:00:00,NaN,NaN,N,46500.0,1083351
296,3612744,1,333788.0,1,1.0,NaN,13836,NaN,NaN,NaN,NaN,NaN,2013-09-01T10:00:00,NaN,NaN,N,46500.0,1083351
297,3612745,1,333788.0,1,1.0,NaN,13461,12.71,12.71,NaN,NaN,13.35,2013-09-01T10:00:00,NaN,NaN,N,46500.0,1083351
298,4892083,2379,447259.0,53,1.0,NaN,18399,6.98,6.98,NaN,NaN,7.33,2017-01-27T18:00:00,NaN,NaN,N,46500.0,1512308
299,4892084,2379,447259.0,53,1.0,NaN,18399,6.98,6.98,NaN,NaN,7.33,2017-01-27T18:00:00,NaN,NaN,N,46500.0,1512308
303,5201255,2575,465973.0,53,1.0,NaN,18372,6.98,6.98,NaN,NaN,7.33,2017-10-14T19:30:00,NaN,NaN,N,46500.0,1598590
304,5201256,2575,465973.0,53,3.0,NaN,18378,5.98,5.98,NaN,NaN,6.28,2017-10-14T19:30:00,NaN,NaN,N,46500.0,1598590


,ticketId__c,saleKey__r.saleId__c,saleDetailKey__r.saleDetailId__c,itemDescription__c,ticketDisplay__c
194,1242739,1083351,3612729,Rocky Mountain Express 2D,TT1242739
195,1242740,1083351,3612730,Rocky Mountain Express 2D,TT1242740
196,1242741,1083351,3612731,Rocky Mountain Express 2D,TT1242741
197,1242742,1083351,3612743,Science Centre Admission,TT1242742
198,1242743,1083351,3612744,Science Centre Admission,TT1242743
199,1242744,1083351,3612745,Science Centre Admission,TT1242744
200,2696199,1512308,4892083,Mysteries of China 3D,TT2696199
201,2696200,1512308,4892084,Mysteries of China 3D,TT2696200
202,3022971,1598590,5201255,Dunkirk (2017): The IMAX Experience,TT3022971
203,3022972,1598590,5201256,Dunkirk (2017): The IMAX Experience,TT3022972


In [ ]:
#more importantly, I missed this with testing...
# test that every 

In [32]:
#| eval : false
sf.write_dict_to_csv(_d )
sf.upload_csv_to_sf(clean_all=True)

Deleting Contact objects from Salesforce
Retrieving Object Ids for Contact from Salesforce
total number of objects =  26
In Salesforce.delete_sf_objects: Deleting 26 Contact objects using /Users/josephmann/Documents/Github/doubledot/sf_download/Contact.csv
execute_job
Salesforce.process_objects: sf_object_s : Contact
Created job 7508Y00000nI5vOQAS for Contact with operation delete
Uploading job 7508Y00000nI5vOQAS of object Contact
job status: InProgress
waiting for job to complete, try 0, status: InProgress
Failed results:
"sf__Id"	"sf__Error"	Id
"0038Y00003bx8WPQAY"	"DELETE_FAILED:Your attempt to delete Ms. Rose Gonzalez could not be completed because it is associated with the following cases.: 00001000
:--"	"0038Y00003bx8WPQAY"
"0038Y00003bx8WQQAY"	"DELETE_FAILED:Your attempt to delete Mr. Sean Forbes could not be completed because it is associated with the following cases.: 00001017, 00001018
:--"	"0038Y00003bx8WQQAY"
"0038Y00003bx8WRQAY"	"DELETE_FAILED:Your attempt to delete Mr. Ja

## Problems ...


In [33]:
#| echo: false
# why does 889832 not have any tickets or saleDetail pointing at it? or me: 82192 323674 (I am 73012)
# why does this point from saleId and not saleKey?
_d['SaleDetail__c']['saleId__r.saleId__c']

292    1083351
293    1083351
294    1083351
295    1083351
296    1083351
297    1083351
298    1512308
299    1512308
303    1598590
304    1598590
377    1718108
378    1723492
379    1728963
380    1738213
381    1738213
384    1765366
385    1769387
386    1779503
387    1810652
388    1811241
389    1834422
Name: saleId__r.saleId__c, dtype: int64